In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from snmfem.conf import RESULTS_PATH, DATASETS_PATH
from snmfem.experiments import load_data
from snmfem.measures import find_min_angle, find_min_MSE
from pathlib import Path
import matplotlib.pyplot as plt
import re
import numpy as np


# Load data and ground truth

In [5]:
file = "smoothnmf_low_tol.npz"
dataset_path = DATASETS_PATH / Path("aspim037_N100_2ptcls_brstlg/sample_4.npz")
path = RESULTS_PATH / Path(file)
data = np.load(path)
X,true_spectra,true_maps,G,shape_2d = load_data(dataset_path)
k = true_spectra.shape[0]
GP = data["GP"]
A = data["A"]
metrics = data["metrics"]
# losses = data["losses"]
angles, true_ind_spec = find_min_angle(true_spectra,GP.T,get_ind=True)
mse, true_ind_map = find_min_MSE(true_maps,A,get_ind=True)

## Plot params

In [6]:
fontsize = 30
scale = 15
aspect_ratio = 1.4
marker_list = ["-o","-s","->","-<","-^","-v","-d"]
mark_space = 20


# Plot spectra and maps

In [7]:
fig, axes = plt.subplots(k,3,figsize = (scale/k * 3 * aspect_ratio,scale))
for i, spectre in enumerate(true_spectra): 
    axes[2,i].plot(true_spectra[i],'bo',label='truth',linewidth=4)
    axes[2,i].plot(GP.T[true_ind_spec[i]] ,'r-',label='reconstructed',markersize=3.5)
    axes[2,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
    axes[2,i].set_xlim(0,1000)

    axes[1,i].imshow(A[true_ind_spec[i]].reshape(shape_2d),vmin = 0.0, vmax = 1.0 )
    axes[1,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    # axes[i,1].set_ylim(0.0,1.0)
    axes[1,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)

    im = axes[0,i].imshow(true_maps[i].reshape(shape_2d),vmin = 0.0, vmax = 1.0)
    axes[0,i].set_title("Phase {}".format(i),fontsize = fontsize)
    axes[0,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
    axes[2,0].legend()

rows = ["True maps","Reconstructed maps","Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.5, 0.01, 0.3])
fig.colorbar(im,cax=cbar_ax)

# fig.tight_layout()

plt.show()

In [54]:
fig, axes = plt.subplots(k,2,figsize = (scale/k * 2 * aspect_ratio,scale))
for i, spectre in enumerate(true_spectra):
    axes[i,0].plot(true_spectra[i],'b-',label='truth',linewidth=3)
    axes[i,0].set_xlim(0,1000)
    
    im = axes[i,1].imshow(true_maps[i].reshape(shape_2d),vmin = 0.0, vmax = 1.0)
    axes[i,1].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
    
axes[0,0].set_title("True spectra",fontsize = fontsize)
axes[0,1].set_title("True maps",fontsize = fontsize)
    
fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.35, 0.01, 0.3])
fig.colorbar(im,cax=cbar_ax)

In [ ]:
np.savetxt("true_spectra.txt",true_spectra)
np.savetxt("GP.txt",GP.T)

In [ ]:
fig, axes = plt.subplots(2,k,figsize = (scale/k * 3 * aspect_ratio,scale))
fig.subplots_adjust(wspace=0.01,hspace=0.01)



for i, spectre in enumerate(true_spectra): 
    # axes[1,i].plot(true_spectra[i],'bo',label='truth',linewidth=4)
    # axes[1,i].plot(GP.T[true_ind_spec[i]] ,'r-',label='reconstructed',markersize=3.5)
    # axes[1,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
    # axes[1,i].set_xlim(0,1000)

    axes[1,i].imshow(A[true_ind_spec[i]].reshape(shape_2d),vmin = 0.0, vmax = 1.0 )
    axes[1,i].set_title("{:.2f} MSE".format(mse[i]),fontsize = fontsize-2)
    axes[1,i].tick_params(axis=u'both', which=u'both',length=0)
    axes[1,i].set_xticklabels([])
    axes[1,i].set_yticklabels([])
    # axes[i,1].set_ylim(0.0,1.0)

    im = axes[0,i].imshow(true_maps[i].reshape(shape_2d),vmin = 0.0, vmax = 1.0)
    axes[0,i].set_title("Phase {}".format(i),fontsize = fontsize)
    axes[0,i].tick_params(axis=u'both', which=u'both',length=0)
    axes[0,i].set_xticklabels([])
    axes[0,i].set_yticklabels([])

    # axes[2,0].legend()

rows = ["Truth","Reconstructed","Spectra"]

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)


fig.subplots_adjust(right=0.84)
# put colorbar at desire position
cbar_ax = fig.add_axes([0.85, 0.4, 0.02, 0.3])
fig.colorbar(im,cax=cbar_ax)
cbar_ax.tick_params(labelsize=fontsize)

# fig.tight_layout()

plt.show()

# Plot losses

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(4*scale,scale))

names = list(losses.dtype.names)
for j,name in enumerate(names) :
    if re.match(r".*(loss)",name) : 
        axes[0].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[0].set_yscale("log")
        axes[0].set_xscale("log")
        axes[0].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[0].set_xscale("log")
        axes[0].legend(fontsize = fontsize -2)
        axes[0].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(rel)",name) : 
        axes[1].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[1].legend(fontsize = fontsize -2)
        axes[1].set_yscale("log")
        axes[1].set_xscale("log")
        axes[1].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[1].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(ang)",name) :
        axes[2].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[2].legend(fontsize = fontsize -2)
        axes[2].set_xscale("log")
        axes[2].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[2].set_xlabel("number of iterations",fontsize = fontsize -5)
    elif re.match(r"^(mse)",name) :
        axes[3].plot(losses[name],marker_list[j%len(marker_list)],markersize=3.5,label = name,markevery = mark_space,linewidth = 2)
        axes[3].legend(fontsize = fontsize -2)
        axes[3].set_xscale("log")
        axes[3].tick_params(axis='both', which='major', labelsize=fontsize-5)
        axes[3].set_xlabel("number of iterations",fontsize = fontsize -5)